In [13]:
import pandas as pd
import ast
import datasets
all  = pd.read_csv(f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/all_review_points.csv')

taken = pd.read_csv(f'../data/taken_1k_samples_human_annotation_sampled.csv')['point']
taken  = pd.concat([taken, pd.read_csv(f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_gathered/all_human_annotations_processed.csv')['review_point']])

In [14]:
print(len(all))
## remove the taken samples from all
all = all[~all['point'].isin(taken)]

207187


In [15]:
len(all)

206119

In [16]:
# Total number of samples
total_samples = 1200

# Uniform sampling
grouped = all.groupby("venue")
samples_per_group = max(1, total_samples // grouped.ngroups)  # Divide total_samples evenly
sampled_df = (
    grouped.apply(lambda group: group.sample(n=min(len(group), samples_per_group), random_state=42))
    .reset_index(drop=True)
)

# If more rows are still needed due to rounding, sample the remainder
remainder = total_samples - len(sampled_df)
if remainder > 0:
    remaining_sample = all.loc[~all.index.isin(sampled_df.index)].sample(n=remainder, random_state=42)
    sampled_df = pd.concat([sampled_df, remaining_sample]).reset_index(drop=True)

/tmp/ipykernel_2199146/2537484748.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(lambda group: group.sample(n=min(len(group), samples_per_group), random_state=42))


In [17]:
sampled_df.value_counts("venue")

venue
ICLR_2025     93
ICLR_2024     88
ACL_2017      80
ARR_2022      80
EMNLP_2023    80
ICLR_2021     80
ICLR_2022     80
ICLR_2023     80
NIPS_2017     80
NIPS_2018     80
NIPS_2019     80
NIPS_2020     80
NIPS_2021     80
NIPS_2022     80
NIPS_2016     59
Name: count, dtype: int64

In [18]:
sampled_df.to_csv(f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/unfiltered_sampled_data.csv', index=False)

In [6]:
# ## add columns: actionability	grounding_specificity	verifiability	helpfulness	appropriateness	Do you need full review?
# sampled_df["actionability"] = ""
# sampled_df["grounding_specificity"] = ""
# sampled_df["verifiability"] = ""
# sampled_df["helpfulness"] = ""
# sampled_df["appropriateness"] = ""
# sampled_df["Do you need full review?"] = ""
# ## load HF authentication token from .env file
# import os
# from dotenv import load_dotenv
# load_dotenv()
# hf_token = os.getenv("HF_TOKEN")
# # Convert DataFrame to Dataset
# ds = datasets.Dataset.from_pandas(sampled_df)

# # Push the dataset to the Hugging Face Hub
# ds.push_to_hub("boda/review_rewrite_test",token=hf_token)

In [14]:
# sampled_df.to_csv(f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/100_point_for_manual_annotation.csv', index=False)